## Языковое моделирование
Задание
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [293]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
import numpy as np
import os
import time

In [294]:
#загружаем файл
text = open('/KAK_Ya_VSTRETIL_VAShU_MAMU_1.txt', 'rb').read().decode(encoding='utf-8')

# Определяем длину текста (количество букв в нем)
print('Length of text: {} characters'.format(len(text)))

Length of text: 56360 characters


In [295]:
#просматриваем фрагмент текста
print(text[:500])

АКТ ПЕРВЫЙ
ИНТ. 2029 ГОСТИНАЯ - ВЕЧЕР
РАССКАЗЧИК, ДОЧЬ, СЫН
Уютный диван в гостиной некоего пригородного дома. В камине
потрескивает огонь. Двое подростков - 14-летний СЫН и 16-
летняя ДОЧЬ - сидят на диване. Они глядят в камеру и слушают
РАССКАЗЧИКА.
                      РАССКАЗЧИК (З.К.)
             Ладно, ребятки, вы уже достаточно
             взрослые, и я собираюсь рассказать
             вам о том, как я встретил вашу
             маму.
                       ДОЧЬ
         


In [296]:
text = text + text + text

In [297]:
# уникальные буквы в файле
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

91 unique characters


In [298]:
#словарь из букв
vocab

['\n',
 '\r',
 ' ',
 '!',
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '«',
 '»',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Щ',
 'Ы',
 'Ь',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 '–',
 '…',
 '№']

In [299]:
# Задаем отображения из множества уникальных букв в множество индексов
char2idx = {u:i for i, u in enumerate(vocab)}  #{уникальная буквы : ее индекс из словаря}

# Представление текста в виде последовательности чисел (индексы букв из словаря)
idx2char = np.array(vocab) # список из символов словаря (можно по индексу извлекать букву)
text_as_int = np.array([char2idx[c] for c in text])

In [300]:
print(text[:500]), print(text_as_int[:500])

АКТ ПЕРВЫЙ
ИНТ. 2029 ГОСТИНАЯ - ВЕЧЕР
РАССКАЗЧИК, ДОЧЬ, СЫН
Уютный диван в гостиной некоего пригородного дома. В камине
потрескивает огонь. Двое подростков - 14-летний СЫН и 16-
летняя ДОЧЬ - сидят на диване. Они глядят в камеру и слушают
РАССКАЗЧИКА.
                      РАССКАЗЧИК (З.К.)
             Ладно, ребятки, вы уже достаточно
             взрослые, и я собираюсь рассказать
             вам о том, как я встретил вашу
             маму.
                       ДОЧЬ
         
[25 35 43  2 40 30 41 27 51 34  1  0 33 38 43  8  2 12 10 12 19  2 28 39
 42 43 33 38 25 55  2  7  2 27 30 48 30 41  1  0 41 25 42 42 35 25 32 48
 33 35  6  2 29 39 48 52  6  2 42 51 38  1  0 44 86 74 69 83 65  2 60 64
 58 56 69  2 58  2 59 70 73 74 64 69 70 65  2 69 61 66 70 61 59 70  2 71
 72 64 59 70 72 70 60 69 70 59 70  2 60 70 68 56  8  2 27  2 66 56 68 64
 69 61  1  0 71 70 74 72 61 73 66 64 58 56 61 74  2 70 59 70 69 84  8  2
 29 58 70 61  2 71 70 60 72 70 73 74 66 70 58  2  7  2 11 14  7 67 61 74
 

(None, None)

In [301]:
# Максимальная длина предложения для входных данных (в буквах)
seq_length = 100

#количество эпох
examples_per_epoch = len(text)//(seq_length+1)

In [302]:
# создаем датасет из данных
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for element in char_dataset:
    print(len(char_dataset))
    print(element)
    break

169080
tf.Tensor(25, shape=(), dtype=int64)


In [303]:
for element in char_dataset:
    print(idx2char[element.numpy()])
    break

А


In [304]:
#создаем батчи из seq_length+1 элементов
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) #drop_remainder удаляет неполный батч (с длиной менее seq_length+1)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'АКТ ПЕРВЫЙ\r\nИНТ. 2029 ГОСТИНАЯ - ВЕЧЕР\r\nРАССКАЗЧИК, ДОЧЬ, СЫН\r\nУютный диван в гостиной некоего пригор'
'одного дома. В камине\r\nпотрескивает огонь. Двое подростков - 14-летний СЫН и 16-\r\nлетняя ДОЧЬ - сидят'
' на диване. Они глядят в камеру и слушают\r\nРАССКАЗЧИКА.\r\n                      РАССКАЗЧИК (З.К.)\r\n   '
'          Ладно, ребятки, вы уже достаточно\r\n             взрослые, и я собираюсь рассказать\r\n       '
'      вам о том, как я встретил вашу\r\n             маму.\r\n                       ДОЧЬ\r\n             О'


In [305]:
# Разбиваем каждый батч на признаки и целевую переменную (последнюю букву)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#применяем функцию split_input_target ко всем батчам
dataset = sequences.map(split_input_target)

In [306]:
for input_example, target_example in dataset:
    print(f'Количество батчей: {len(dataset)}')
    print(f'Данные: {input_example}')
    print(f'Целевая переменная: {target_example}')
    break

Количество батчей: 1674
Данные: [25 35 43  2 40 30 41 27 51 34  1  0 33 38 43  8  2 12 10 12 19  2 28 39
 42 43 33 38 25 55  2  7  2 27 30 48 30 41  1  0 41 25 42 42 35 25 32 48
 33 35  6  2 29 39 48 52  6  2 42 51 38  1  0 44 86 74 69 83 65  2 60 64
 58 56 69  2 58  2 59 70 73 74 64 69 70 65  2 69 61 66 70 61 59 70  2 71
 72 64 59 70]
Целевая переменная: [35 43  2 40 30 41 27 51 34  1  0 33 38 43  8  2 12 10 12 19  2 28 39 42
 43 33 38 25 55  2  7  2 27 30 48 30 41  1  0 41 25 42 42 35 25 32 48 33
 35  6  2 29 39 48 52  6  2 42 51 38  1  0 44 86 74 69 83 65  2 60 64 58
 56 69  2 58  2 59 70 73 74 64 69 70 65  2 69 61 66 70 61 59 70  2 71 72
 64 59 70 72]


In [307]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'АКТ ПЕРВЫЙ\r\nИНТ. 2029 ГОСТИНАЯ - ВЕЧЕР\r\nРАССКАЗЧИК, ДОЧЬ, СЫН\r\nУютный диван в гостиной некоего приго'
Target data: 'КТ ПЕРВЫЙ\r\nИНТ. 2029 ГОСТИНАЯ - ВЕЧЕР\r\nРАССКАЗЧИК, ДОЧЬ, СЫН\r\nУютный диван в гостиной некоего пригор'


In [308]:
# размер батча
BATCH_SIZE = 64

# размер буфера для перемешивания данных
BUFFER_SIZE = 10000

# перемешивание разделенных данных
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [309]:
# Длина словаря в буквах
vocab_size = len(vocab)

# Размерность эмбеддинга
embedding_dim = 256

# Число ячеек
rnn_units = 1024

In [313]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [314]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [315]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 91) # (batch_size, sequence_length, vocab_size)


In [316]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (64, None, 256)           23296     
                                                                 
 lstm_69 (LSTM)              (64, None, 1024)          5246976   
                                                                 
 lstm_70 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_71 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_72 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 dense_19 (Dense)            (64, None, 91)            93275     
                                                                 
Total params: 30,541,659
Trainable params: 30,541,659

In [317]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 91), dtype=float32, numpy=
array([[-1.5729660e-06, -1.4143723e-05,  9.8893797e-06, ...,
        -3.1643076e-06,  1.1133703e-05,  1.7663368e-05],
       [-9.2694904e-07, -3.5263216e-05,  2.0985739e-05, ...,
        -1.0136719e-06,  3.7269318e-05,  5.9394595e-05],
       [ 5.4163297e-06, -5.1699772e-05,  5.1134892e-05, ...,
        -1.6890990e-06,  7.9410427e-05,  1.2162144e-04],
       ...,
       [ 2.1410463e-03, -9.5573079e-05, -6.5407564e-04, ...,
        -8.0047984e-04,  3.3273671e-03,  1.0984152e-03],
       [ 1.9887080e-03,  2.5286339e-05, -7.9202000e-04, ...,
        -8.7619666e-04,  3.3234609e-03,  1.1696751e-03],
       [ 1.7625657e-03,  9.0853428e-05, -9.2750683e-04, ...,
        -9.5121033e-04,  3.2700037e-03,  1.2351001e-03]], dtype=float32)>

In [318]:
#индексы букв (извлекаем выборки из категориального распределения)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

#убираем лишнюю размерность (список индексов)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([56, 75, 24, 28, 25,  7, 82, 10, 39, 33, 62, 18, 64, 38, 39, 25,  1,
       58, 57, 46, 26, 63, 59, 30, 39, 23, 82, 81, 15, 61, 46, 55, 73, 71,
       27, 27, 42, 71, 64, 10, 71, 42, 26, 29, 57, 13, 50, 68, 71, 75, 74,
        0, 69, 78, 49, 90, 71, 25, 18,  2, 21, 77, 33, 51, 85,  4,  2, 75,
       49, 77,  8, 74, 47, 56, 88,  1,  2, 60, 70, 81, 11, 21, 30, 78, 59,
       19, 86, 21, 36, 29, 24, 75, 67, 18, 37, 36, 71, 47, 58, 60])

In [319]:
#входной пример
#возвращаем строку
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()

#предсказанная буква
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'Маршала и грубо заковывает в наручники.\r\nКВАРТИРА РОБИН - НОЧЬ\r\nТэд и Робин стоят у двери, прощаясь.'

Next Char Predictions: 
 'ау»ГА-ъ0ОИж8иНОА\rвбХБзгЕО«ъщ5еХЯспВВСпи0пСБДб3Щмпут\nнцШ№пА8 ;хИЫэ( уШх.тЦа–\r дощ1;Ецг9ю;ЛД»ул8МЛпЦвд'


In [320]:
#функция потерь
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 91)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.511663


In [321]:
#компиляция модели
model.compile(optimizer='adam', loss=loss)

In [322]:
# место для хранения checkpoint
checkpoint_dir = '/content/drive/MyDrive/Check'
# Имя файла checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

In [323]:
#обучение модели
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
26/26 [==============================] - 13s 316ms/step - loss: 3.1812
Epoch 2/100
26/26 [==============================] - 9s 318ms/step - loss: 2.8023
Epoch 3/100
26/26 [==============================] - 9s 321ms/step - loss: 2.4944
Epoch 4/100
26/26 [==============================] - 9s 324ms/step - loss: 2.2800
Epoch 5/100
26/26 [==============================] - 9s 327ms/step - loss: 2.0448
Epoch 6/100
26/26 [==============================] - 9s 329ms/step - loss: 1.8607
Epoch 7/100
26/26 [==============================] - 9s 332ms/step - loss: 1.7790
Epoch 8/100
26/26 [==============================] - 9s 332ms/step - loss: 1.7095
Epoch 9/100
26/26 [==============================] - 9s 334ms/step - loss: 1.6261
Epoch 10/100
26/26 [==============================] - 9s 336ms/step - loss: 1.5626
Epoch 11/100
26/26 [==============================] - 9s 338ms/step - loss: 1.5320
Epoch 12/100
26/26 [==============================] - 9s 338ms/step - loss: 1.4508
Epoch 13/100

In [324]:
#предсказание модели
example_batch_predictions = model(input_example_batch)
print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 91) # (batch_size, sequence_length, vocab_size)


In [325]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [326]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'Маршала и грубо заковывает в наручники.\r\nКВАРТИРА РОБИН - НОЧЬ\r\nТэд и Робин стоят у двери, прощаясь.'

Next Char Predictions: 
 'сршале.и  рубо заковывает в наручники.\r\nКмАРТИРА РОБИН - НОЧЬ\r\nТэд и Робин стоят у двори, прощаясь.\r'


In [327]:
#Находим имя файла последней сохраненной контрольной точки
tf.train.latest_checkpoint(checkpoint_dir)

'/content/drive/MyDrive/Check/ckpt_85'

In [328]:
#строим модель
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#загружаем веса из последней сохраненной контрольной точки в модель
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [329]:
#информация о модели
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (1, None, 256)            23296     
                                                                 
 lstm_73 (LSTM)              (1, None, 1024)           5246976   
                                                                 
 lstm_74 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_75 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_76 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_20 (Dense)            (1, None, 91)             93275     
                                                                 
Total params: 30,541,659
Trainable params: 30,541,659

In [334]:
def generate_text(model, start_string):
    # Этап оценки (генерация текста с использованием обученной модели)

    # число букв для генераци
    num_generate = 500

    # Преобразование начальной строки в числа (векторизация)
    input_eval = [char2idx[s] for s in start_string]
    #Возвращаем тензор с осью длины 1, вставленной первой в индекс
    input_eval = tf.expand_dims(input_eval, 0)

    # Пустая строка для хранения результатов
    text_generated = []

    # Низкая температура приводит к более предсказуемому тексту.
    # Более высокая температура приводит к более неожиданному тексту.
    temperature = 0.001

    # здесь batch size == 1
    # сбрасываем состояния всех слоев в модели
    model.reset_states()
    for i in range(num_generate):
        #получаем предсказания модели
        predictions = model(input_eval)
        #удаляем первую размерность в предсказании
        predictions = tf.squeeze(predictions, 0)
        # использование категориального распределения для прогнозирования символа, возвращаемого моделью
        predictions = predictions / temperature
        #извлекаем выборку из категориального распределения
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Передаем предсказанный символ в качестве следующего ввода в модель
        # вместе с предыдущим скрытым состоянием 
        # добавляем 1 первым индексом к размерности
        input_eval = tf.expand_dims([predicted_id], 0)

        #сохраняем предсказанную букву
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [335]:
text_ = generate_text(model, start_string=u"Привет, как дела? ")
print(text_)

Привет, как дела? Он смотрит, кто это
и отвечает.
                       БАРНИ
             Эй, ты же знаешь, как я всегда
             тащился от полуазиатских девчонок?
СТОП-КАДР НА БАРНИ.
                    РОБИН
          Знаешь, думаю, мне нравится твоя
          теория оливок в моем
          холодильнике уже целую вечность
          тусуется.
                    ТЭД
              (в шоке)
          Ты ешь оливки?
                    ТЭД
              (пауза)
          Ага, я никогда не пой
